<a href="https://colab.research.google.com/github/vvjft/IQA-CNNpp/blob/main/iqa_cnn%2B%2B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install unrar
#import logging # can be set once for session
#logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', datefmt='%H:%M')

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unrar is already the newest version (1:6.1.5-1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [18]:
import numpy as np
import pandas as pd
import cv2
import subprocess
import os
from scipy.signal import convolve2d
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
import sys

class database_loader:
    ''' Parent class for database-specific loaders '''
    def __init__(self):
      self.catalogue = 'databases'
      ### Attributes to be declared within the child class ###
      self.url = ''      # URL of the dataset
      self.dir = ''      # Directory where the exctracted dataset is stored
      self.images = ''   # Directory where the images are stored
      self.rar_file = '' # Path to the rar file
      self.zip_file = '' # Path to the zip file

    def download(self, file, exctract_in='databases'):
        try:
            #!wget {self.url} -P {self.catalogue}
            if file.endswith('.rar'):
                !unrar x -inul {file} {exctract_in}
            elif file.endswith('.zip'):
                !unzip -q {file} -d {exctract_in}
            else:
                print(f"Unsupported file type: {file}")
                return False
            print(f"Dataset extracted in '{exctract_in}'")
        except Exception as e:
            print(f"Failed to download dataset: {e}.")
            return False

    def patch_files_exist(self):
        '''Check if patch files are present in the directory.'''
        return (os.path.exists(os.path.join(self.dir, 'normalized_distorted_images', 'training', 'patch_training.csv')) and
                os.path.exists(os.path.join(self.dir, 'normalized_distorted_images', 'validation', 'patch_validation.csv')) and
                os.path.exists(os.path.join(self.dir, 'normalized_distorted_images', 'test', 'patch_test.csv')))

    def save_data(self, data, set_type):
        '''Save the data to disk.'''
        X, y = data
        np.save(os.path.join(self.dir, f'X_{set_type}.npy'), X)
        np.save(os.path.join(self.dir, f'y_{set_type}.npy'), y)
        print(f"{set_type} data saved successfully.")

    def load_data(self, set_type):
        '''Load the data from disk.'''
        X = np.load(os.path.join(self.dir, f'X_{set_type}.npy'))
        y = np.load(os.path.join(self.dir, f'y_{set_type}.npy'))
        return X, y

    def data_exist(self):
        '''Check if patch files are present in the directory.'''
        return (os.path.exists(os.path.join(self.dir, 'X_train.npy')) and os.path.exists(os.path.join(self.dir, 'y_train.npy')) and
                os.path.exists(os.path.join(self.dir, 'X_val.npy')) and os.path.exists(os.path.join(self.dir, 'y_val.npy')) and
                os.path.exists(os.path.join(self.dir, 'X_test.npy')) and os.path.exists(os.path.join(self.dir, 'y_test.npy')))

    def split_data(self, data1):
        train_data, test_data = train_test_split(data1, test_size=0.2, random_state=40)
        train_data, val_data = train_test_split(train_data, test_size=0.25, random_state=40)
        return train_data, val_data, test_data

    def preprocess(self, train_data, val_data, test_data, patch_size=32):

        def normalize_image(patch, P=3, Q=3, C=1):
            kernel = np.ones((P, Q)) / (P * Q)
            patch_mean = convolve2d(patch, kernel, boundary='symm', mode='same')
            patch_sm = convolve2d(np.square(patch), kernel, boundary='symm', mode='same')
            patch_std = np.sqrt(np.maximum(patch_sm - np.square(patch_mean), 0)) + C
            patch_ln = (patch - patch_mean) / patch_std
            return patch_ln.astype('float32')

        def slice_image(image, patch_size=32):
            height, width = image.shape[:2]
            num_patches_y = height // patch_size
            num_patches_x = width // patch_size
            patch_count = 0
            for i in range(num_patches_y):
                for j in range(num_patches_x):
                    patch = image[i*patch_size:(i+1)*patch_size, j*patch_size:(j+1)*patch_size]
                    patch_path = os.path.join(output_dir_patches, f"{os.path.splitext(filename)[0]}_patch_{patch_count}.bmp")
                    patch_filename = f"{os.path.splitext(filename)[0]}_patch_{patch_count}.bmp"
                    cv2.imwrite(patch_path, patch)
                    self.patches.append([patch_filename, mos, distortion])
                    patch_count += 1

        sets = [(train_data, 'training'), (val_data, 'validation'), (test_data, 'test')]
        dfs = []
        total_images = sum(len(data) for data, _ in sets)
        processed_images = 0
        print('Preprocessing images...')

        for (data, name) in sets:
            output_dir_full = os.path.join(self.dir, 'normalized_distorted_images', name, 'full')
            output_dir_patches = os.path.join(self.dir, 'normalized_distorted_images', name, 'patches')
            os.makedirs(output_dir_full, exist_ok=True)
            os.makedirs(output_dir_patches, exist_ok=True)
            self.patches = []
            for row in data.itertuples(index=False):
                filename = row[0]
                mos = row[1]
                distortion = row[2]
                image_path = os.path.join(self.images, filename)
                image = cv2.imread(image_path)
                if image is None:
                    print(f"Failed to load image: {filename}")
                    continue
                image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                image_normalized = normalize_image(image_gray)
                filename = f'NORM_{filename.lower()}'
                cv2.imwrite(os.path.join(output_dir_full, filename), image_normalized)
                slice_image(image_normalized, patch_size)
                processed_images += 1
                sys.stdout.write(f'\rProcessed {processed_images}/{total_images} images. ')
                sys.stdout.flush()

            patches = pd.DataFrame(self.patches, columns=['image', 'MOS', 'distortion'])
            dfs.append(patches)
        return dfs

    def encode(self, dataframes):
        '''Encodes distortion labels into one-hot vectors.'''
        for i in range(len(dataframes)):
            dists = dataframes[i]['distortion']
            le = LabelEncoder()
            y_class_encoded = le.fit_transform(dists)
            dists_one_hot = to_categorical(y_class_encoded, num_classes=13).astype(int)
            dataframes[i]['distortion_encoded'] = [np.array(one_hot) for one_hot in dists_one_hot]
            dataframes[i] = dataframes[i].drop(['distortion'], axis=1)
        return dataframes

    def map2tf(self, set_type, data):
        '''
        Maps data into format excected by TensorFlow: adds chanel dimension and stores data in arrays.
        set_type: 'training', 'validation', 'test'
        data: DataFrame with columns: 'image', 'MOS', 'distortion'
        '''
        images_dir = os.path.join(self.dir, 'normalized_distorted_images', set_type, 'patches')
        X, y = [], []
        for row in data.itertuples(index=False):
            filename = row[0]
            score = row[1]
            file_path = os.path.join(images_dir, filename)
            if filename.endswith(('.bmp', '.png')) and os.path.exists(file_path):
                img = cv2.imread(file_path, cv2.IMREAD_UNCHANGED)
                X.append(img)
                y.append(score)
            else:
                print(f"File not found: {file_path}")
        X = np.array(X)
        y = np.array(y)
        X = X[..., np.newaxis]
        return X, y

class tid2013_loader(database_loader):
    def __init__(self, download=True):
        super().__init__()
        self.url = 'https://www.ponomarenko.info/tid2013/tid2013.rar'
        self.dir = os.path.join(self.catalogue, 'tid2013')
        self.images = os.path.join(self.dir, 'distorted_images')
        self.rar_file = os.path.join(self.catalogue, 'tid2013.rar')
        self.distortion_mapping = {1: 'wn', 2:'wnc', 3:'scn', 4:'mn', 5:'hfn', 6:'in', 7:'qn', 8: 'gblur', 9:'idn', 10: 'jpeg', 11: 'jp2k', 12:'jpegte', 13:'jp2kte'} # According to TID2013 documentation

        os.makedirs(self.dir, exist_ok=True)
        if download:
            self.download(self.rar_file, exctract_in=self.dir)
        elif not os.path.exists(self.rar_file):
            print("Dataset not found. Downloading...")
            self.download(self.url, self.rar_file)

        if self.data_exist():
            print("Patch files found. Loading patched data...")
            self.train = self.load_data('train')
            self.val = self.load_data('val')
            self.test= self.load_data('test')
        else:
            data = self.prepare_data()
            print('Mapping data to TensorFlow format...')
            self.train = self.map2tf('training', data[0])
            self.val = self.map2tf('validation', data[1])
            self.test = self.map2tf('test', data[2])
            self.save_data(self.train, 'train')
            self.save_data(self.val, 'val')
            self.save_data(self.test, 'test')
        #self.train, self.val, self.test = self.encode(data)

        print("Data loaded successfully.")

    def prepare_data(self, filter=True):
        data_path = os.path.join(self.dir, 'mos_with_names.txt')
        data = pd.read_csv(data_path, header=None, delimiter=' ')
        data = data.iloc[:, [1, 0]]  # swap column order
        data.columns = ['image', 'MOS']
        data['distortion'] = data['image'].apply(lambda x: self.distortion_mapping.get(int(x.split('_')[1]), 'other'))
        if filter:
            data = data[data['distortion'].isin(self.distortion_mapping.values())]
        data.to_csv(os.path.join(self.dir,'mos_with_names.csv'), index=False)

        train_data, val_data, test_data = self.split_data(data)
        train_data, val_data, test_data = self.preprocess(train_data, val_data, test_data)

        return [train_data, val_data, test_data]

class kadid10k_loader(database_loader):
    def __init__(self, download=True):
        super().__init__()
        self.url = 'https://datasets.vqa.mmsp-kn.de/archives/kadid10k.zip'
        self.dir = os.path.join(self.catalogue, 'kadid10k')
        self.images = os.path.join(self.dir, 'images')
        self.zip_file = os.path.join(self.catalogue, 'kadid10k.zip')
        self.distortion_mapping = {1: 'gblur', 2: 'lblur', 3: 'mblur', 4: 'cdiff', 5: 'cshift', # According to KADID-10k documentation
                                   6: 'cquant', 7: 'csat1', 8: 'csat2', 9: 'jp2k', 10: 'jpeg',
                                   11: 'wniose1', 12: 'wniose2', 13: 'inoise', 14: 'mnoise', 15: 'denoise',
                                   16: 'bright', 17: 'dark', 18: 'meanshft', 19: 'jit', 20: 'patch',
                                   21: 'pixel', 22: 'quant', 23: 'cblock', 24: 'sharp', 25: 'contrst'}
        if download:
            self.download(self.zip_file)
        elif not os.path.exists(self.zip_file):
            print("Dataset not found. Downloading...")
            self.download(self.url, self.zip_file)

        if self.data_exist():
            print("Patch files found. Loading patched data...")
            self.train = self.load_data('train')
            self.val = self.load_data('val')
            self.test= self.load_data('test')
        else:
            data = self.prepare_data()
            print('Mapping data to TensorFlow format...')
            self.train = self.map2tf('training', data[0])
            self.val = self.map2tf('validation', data[1])
            self.test = self.map2tf('test', data[2])
            self.save_data(self.train, 'train')
            self.save_data(self.val, 'val')
            self.save_data(self.test, 'test')
        #self.train, self.val, self.test = self.encode(data)

        print("Data loaded successfully.")

    def prepare_data(self, filter=True):
        data_path = os.path.join(self.dir, 'dmos.csv')
        data = pd.read_csv(data_path, header=0, usecols=[0, 2])
        data.columns = ['image', 'DMOS']
        data['distortion'] = data['image'].apply(lambda x: self.distortion_mapping.get(int(x.split('_')[1]), 'other'))
        #if filter:
            #data = data[data['distortion'].isin(self.distortion_mapping.values())]
        data.to_csv(os.path.join(self.dir,'dmos_with_names.csv'), index=False)

        train_data, val_data, test_data = self.split_data(data)
        train_data, val_data, test_data = self.preprocess(train_data, val_data, test_data)

        return [train_data, val_data, test_data]

If the download is slow, stop it, delete databases directory (!rm)
 and run it again.

In [19]:
!rm -r 'databases/tid2013'
!rm -r 'databases/kadid10k'
#tid2013_loader = tid2013_loader(download=True)
kadid10k_loader = kadid10k_loader(download=True)

rm: cannot remove 'databases/tid2013': No such file or directory
Dataset extracted in 'databases'
Preprocessing images...
Processed 10125/10125 images.Mapping data to TensorFlow format...
train data saved successfully.
val data saved successfully.
test data saved successfully.
Data loaded successfully.


In [7]:
X_train, y_train = tid2013_loader.train
X_val, y_val = tid2013_loader.val
X_test, y_test = tid2013_loader.test

In [20]:
X_train, y_train = kadid10k_loader.train
X_val, y_val = kadid10k_loader.val
X_test, y_test = kadid10k_loader.test

In [25]:
import tensorflow as tf
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Input(shape=(32, 32, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='linear')
])

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/10
36450/36450 ━━━━━━━━━━━━━━━━━━━━ 546s 15ms/step - loss: 1.2568 - mae: 0.9344 - val_loss: 1.0603 - val_mae: 0.8829
Epoch 2/10
36450/36450 ━━━━━━━━━━━━━━━━━━━━ 530s 14ms/step - loss: 1.0218 - mae: 0.8637 - val_loss: 1.0408 - val_mae: 0.8716
Epoch 3/10
36450/36450 ━━━━━━━━━━━━━━━━━━━━ 585s 15ms/step - loss: 1.0051 - mae: 0.8536 - val_loss: 1.0265 - val_mae: 0.8667
Epoch 4/10
36450/36450 ━━━━━━━━━━━━━━━━━━━━ 566s 15ms/step - loss: 0.9976 - mae: 0.8495 - val_loss: 1.0735 - val_mae: 0.8868
Epoch 5/10
36450/36450 ━━━━━━━━━━━━━━━━━━━━ 540s 15ms/step - loss: 0.9906 - mae: 0.8458 - val_loss: 1.0210 - val_mae: 0.8628
Epoch 6/10
36450/36450 ━━━━━━━━━━━━━━━━━━━━ 533s 14ms/step - loss: 0.9880 - mae: 0.8441 - val_loss: 1.0330 - val_mae: 0.8665
Epoch 7/10
36450/36450 ━━━━━━━━━━━━━━━━━━━━ 565s 14ms/step - loss: 0.9840 - mae: 0.8420 - val_loss: 1.0233 - val_mae: 0.8622
Epoch 8/10
36450/36450 ━━━━━━━━━━━━━━━━━━━━ 540s 15ms/step - loss: 0.9810 - mae: 0.8402 - val_loss: 1.0234 - val_mae: 0.8615


In [26]:
loss, mae = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', loss)
print('Test MAE:', mae)

Test loss: 0.9741964936256409
Test MAE: 0.8370321393013
